In [63]:
import requests
from datetime import datetime
from bs4 import BeautifulSoup
from math import nan
import json, re, os

headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36"}

def render_html(url):
    req = requests.get(url,headers=headers)
    return BeautifulSoup(req.text,"html.parser")

In [64]:
def get_max(page):
    pagination = page.find("div",id="pagination")
    numbers = []
    for tag in pagination.find_all("a",class_="page-numbers"):
        try:
            number = int(tag.get_text())
            numbers.append(number)
        except: pass
    max_page = max(numbers)
    return max_page

In [74]:
def get_jobs(url,include=None):
    jobs = []
    page = render_html(url)
    for a in page.find_all("a",class_=["job-block","d-block"]):
        try:
            try: link = a['data-url']
            except: link = nan
            try: title = a['data-title']
            except: title = nan
            try: city = a['data-location']
            except: city = nan
            try: wage = a.find("span").find_next("span").get_text().strip()
            except: wage = nan
            try: hours = a['data-hours']
            except: hours = nan
            try: type_ = a['data-industry']
            except: type_ = nan
            job = {
                "Link": link,
                "Title": title,
                "City": city,
                "Wage": wage,
                "Hours": hours,
                "Type": type_
            }
            jobs.append(job)
        except: pass    
    if include == "max_page":
        max_page = get_max(page)
        return jobs, max_page
    else:
        return jobs

In [47]:
url = "https://www.covebo.pl/oferty-pracy/?_page=1"
page = render_html(url)

In [ ]:
# for a in page.find_all("a",class_=["job-block","d-block"]):
#     try:
#         link = a['data-url']
#         title = a['data-title']
#         city = a['data-location']
#         wage = a.find("span").find_next("span").get_text().strip()
#         hours = a['data-hours']
#         type_ = a['data-industry']
#         job = {
#             "Link": link,
#             "Title": title,
#             "City": city,
#             "Wage": wage,
#             "Hours": hours,
#             "Type": type_
#         }
#         print(job)
#         print("-"*150)
#     except:
#         print(a)
#         print("-"*150)
#     # print(a,"\n")

In [ ]:
# pagination = page.find("div",id="pagination")
# numbers = []
# for tag in pagination.find_all("a",class_="page-numbers"):
#     try:
#         number = int(tag.get_text())
#         numbers.append(number)
#     except: pass

# max_page = max(numbers)
# max_page

In [75]:
url = "https://www.covebo.pl/oferty-pracy/?_page=1"
jobs, max_page = get_jobs(url,include="max_page")
for i in range(2,max_page+1):
    url = f"https://www.covebo.pl/oferty-pracy/?_page={i}"
    new_jobs = get_jobs(url)
    jobs.extend(new_jobs)
    print(f"{i}/{len(jobs)}",end=",")
print("\b.")

2/32,3/48,4/64,5/80,6/96,7/112,8/128,9/144,10/160,11/176,12/192,13/208,14/224,15/240,16/256,17/272,18/288.


In [ ]:
jobs

In [81]:
import pandas as pd

df = pd.DataFrame(jobs)
df = df.dropna()
df.to_excel("example.xlsx",sheet_name="First",index=False,encoding="utf-8")